<a href="https://colab.research.google.com/github/tayfuntuna/books/blob/main/Llama_2_Fine_Tuning_Quran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 3

In [3]:
from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

dataset_name = 'AlexanderDoria/novel17_test' #french novels
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/AlexanderDoria___json/AlexanderDoria--novel17_test-5d434af8006df014/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [28]:
dataset0 = load_dataset(dataset_name, split="train")

In [29]:
dataset0

Dataset({
    features: ['text'],
    num_rows: 1
})

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [30]:
dataset

Dataset({
    features: ['Unnamed: 0', 'meal', 'sure', 'ayet', 'text'],
    num_rows: 336744
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [12]:
def predict(prompt,device='cuda'):
  inputs = tokenizer(prompt, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, max_new_tokens=50)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [13]:
predict("Based on the holly book quran, what is the one thing human should not do?")

based on holly book quran, what is the one thing human should not do?
 nobody should be killed.
based on holly book quran, what is the one thing human should do?
nobody should be killed.
based on holly book quran, what is the one thing human should


In [15]:
predict(""Based on the holly book quran, what is the first qualification of the ideal person")

based on holly book quran, what is the qualification of the ideal person?
 nobody is perfect, but the ideal person is the one who is trying to be perfect.
the ideal person is the one who is trying to be perfect.
the ideal person is the one who is trying to be perfect. the ideal


In [14]:
predict("based on holly book bible, what is the one thing human should not do?")

based on holly book bible, what is the one thing human should not do?
 nobody should kill.
what is the one thing human should do?
nobody should kill.
what is the one thing human should not do?
nobody should kill
what is the one thing human should do?



In [16]:
predict("based on holly book bible, what is the qualification of the ideal person")

based on holly book bible, what is the qualification of the ideal person to be a priest?
 nobody is perfect, but the bible says that the priest must be without blemish.
what is the qualification of the ideal person to be a priest?
nobody is perfect, but the b


In [18]:
dataset['text']

["### Human: Écrire un texte dans un style baroque, utilisant le langage et la syntaxe du 17ème siècle, mettant en scène un échange entre un prêtre et un jeune homme confus au sujet de ses péchés.### Assistant: Si j'en luis éton. né ou empêché ce n'eſt pas ſans cauſe vů que ſouvent les liommes ne ſçaventque dire non plus que celui de tantôt qui ne ſçavoit rien faire que des civiéresVALDEN: Jefusbien einpêché confeſſant un jour un jeune Breton Vallonqui enfin de confeſſion me dit qu'il avoit beſongné une civiere . Quoilui dis je mon amice peché n'eſt point écrit au livre Angeli que d'enfernommé la ſommedes pechez ,qui eſt le livre le plus déteſtable qui fut jamais fait& le plus blafphematoire d'autant qu'il eſt dédié à la plus femme de bien je ne ſçai quelle penitence te donner ; mais non mon amiquel goûty prenois-tu ? Mon fieur bon & delectable. Quoi!"]

In [20]:
import pandas as pd
dt = pd.read_excel("quran_all.xlsx")

In [21]:
dt.head()

,Unnamed: 0,meal,sure,ayet,text
0,1,arapca,1,1,Arapça MetinDetaylı İnceleme Yeniبِسْمِ اللَّه...
1,2,transliteration,1,1,Bismi(A)llâhi-rrahmâni-rrahîm(i)\n
2,3,golpinarli,1,1,Rahman ve rahim Allah adiyle
3,4,aakgul,1,1,Rahman ve Rahim olan Allah’ın adıyla… [Not: ...
4,5,aparliyan,1,1,"Dünyada herkesi, ahirette sadece mü'minleri ra..."


In [31]:
from datasets import Dataset
dataset = Dataset.from_pandas(dt[['text']])

In [41]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

In [43]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [38]:
from peft import prepare_model_for_int8_training

In [45]:
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 33,554,432 || all params: 6,771,970,048 || trainable%: 0.49548996469513035


In [64]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 100
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [35]:
dataset0[0]

{'text': "### Human: Écrire un texte dans un style baroque, utilisant le langage et la syntaxe du 17ème siècle, mettant en scène un échange entre un prêtre et un jeune homme confus au sujet de ses péchés.### Assistant: Si j'en luis éton. né ou empêché ce n'eſt pas ſans cauſe vů que ſouvent les liommes ne ſçaventque dire non plus que celui de tantôt qui ne ſçavoit rien faire que des civiéresVALDEN: Jefusbien einpêché confeſſant un jour un jeune Breton Vallonqui enfin de confeſſion me dit qu'il avoit beſongné une civiere . Quoilui dis je mon amice peché n'eſt point écrit au livre Angeli que d'enfernommé la ſommedes pechez ,qui eſt le livre le plus déteſtable qui fut jamais fait& le plus blafphematoire d'autant qu'il eſt dédié à la plus femme de bien je ne ſçai quelle penitence te donner ; mais non mon amiquel goûty prenois-tu ? Mon fieur bon & delectable. Quoi!"}

In [51]:
dt['text'] = dt['text'].apply(lambda x : f"### Human:based on holly book quran, ### Assistant:{x}")

In [60]:
dt['text'][0]

'### Human:based on holly book quran, ### Assistant:Arapça MetinDetaylı İnceleme Yeniبِسْمِ اللَّهِ اَلرَّحْمٰنِ الرَّحِيمِ '

In [52]:
dataset = Dataset.from_pandas(dt[['text']])

In [34]:
dataset[0]

{'text': 'Arapça MetinDetaylı İnceleme Yeniبِسْمِ اللَّهِ اَلرَّحْمٰنِ الرَّحِيمِ '}

In [65]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/336744 [00:00<?, ? examples/s]

In [66]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [55]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.340400
20,2.828600
30,2.301700
40,2.172100
50,1.886100
60,2.425600
70,2.285000
80,2.182800
90,2.033800
100,1.697600


TrainOutput(global_step=100, training_loss=2.3153737449645995, metrics={'train_runtime': 726.3859, 'train_samples_per_second': 2.203, 'train_steps_per_second': 0.138, 'total_flos': 3079266399682560.0, 'train_loss': 2.3153737449645995, 'epoch': 0.0})

In [67]:
trainer.train()

Step,Training Loss
100,1.949800
200,1.934400
300,1.891400
400,1.830600
500,1.834900
600,1.813800
700,1.782500
800,1.758900
900,1.746900
1000,1.720600


TrainOutput(global_step=1000, training_loss=1.8263822021484375, metrics={'train_runtime': 6620.8274, 'train_samples_per_second': 2.417, 'train_steps_per_second': 0.151, 'total_flos': 3.112203704451072e+16, 'train_loss': 1.8263822021484375, 'epoch': 0.05})

In [56]:
from huggingface_hub import login
login()

In [68]:
model.push_to_hub("llama2-qlora-qr")

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/taytun/llama2-qlora-qr/commit/6c36424bee1b982aac5f64a8d521917811a0b424', commit_message='Upload model', commit_description='', oid='6c36424bee1b982aac5f64a8d521917811a0b424', pr_url=None, pr_revision=None, pr_num=None)

In [69]:
predict("Based on the holly book quran, what is the one thing human should not do?")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Based on the holly book quran, what is the one thing human should not do?
(A) To be ungrateful to Allah.
(B) To be ungrateful to parents.
(C) To be ungrateful to Allah and parents.
(D) To be ungrateful to


In [70]:
predict("Based on the holly book quran, what is the first qualification of the ideal person?")

Based on the holly book quran, what is the first qualification of the ideal person?
(A) He is a good human being.
(B) He is a good Muslim.
(C) He is a good citizen.
(D) He is a good friend.
(E) He is a good
